In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import holoviews as hv
from holoviews import opts, dim
from gensim.models import LdaModel

In [2]:
# Optimal number of topics
nTopics = 21

In [3]:
# Define colors to associate with each topic
custom_colors = {
 'burlywood': '#DEB887',
 'chocolate': '#D2691E',
 'crimson': '#DC143C',
 'darkgreen': '#006400',
 'darkorange': '#FF8C00',
 'darkslategrey': '#2F4F4F',
 'deepskyblue': '#00BFFF',
 'dimgray': '#696969',
 'firebrick': '#B22222',
 'gold': '#FFD700',
 'goldenrod': '#DAA520',
 'lawngreen': '#7CFC00',
 'lightcoral': '#F08080',
 'lightpink': '#FFB6C1',
 'mediumvioletred': '#C71585',
 'orangered': '#FF4500',
 'orchid': '#DA70D6',
 'royalblue': '#4169E1',
 'slateblue': '#6A5ACD',
 'springgreen': '#00FF7F',
 'steelblue': '#4682B4',
 'teal': '#008080',
 'turquoise': '#40E0D0',
 'yellow': '#FFFF00',
 'blueviolet': '#8A2BE2',
 'yellowgreen': '#9ACD32'}

# turn into a list
colorlist = []
for color in custom_colors.values():
    colorlist.append(tuple(int(color.lstrip('#')[i:i+2], 16) for i in (0, 2, 4)))

In [4]:
# Define topic names
topic_names = [
                'Systems Hydrology',
                'Stochastic Hydrology',
                'Water Temperature',
                'Water Resources Management',
                'Climate Change',
                'Fate & Transport',
                'Sediment & Erosion',
                'Land Surface & Hydromet',
                'Vadose Zone',
                'Soil Moisture',
                'Snow Hydrology',
                'Water Treatment',
                'Groundwater',
                'Channel Flow',
                'Hydrogeology (Modeling?)',
                'Watershed Hydrology',
                'Surface Hydrology Modeling',
                'Numerical Modeling',
                'Chemistry',
                'Computaional Hydrology',
                'Organic Waste and Pollutants',
                'Topic 21',
                'Topic 22',
                'Topic 23',
                'Topic 24',
                'Topic 25',
                'Topic 26',
                'Topic 27',
                'Topic 28',
                'Topic 29',
                'Topic 30',
                ]

# Load Data

In [5]:
# Load model
lda_model = LdaModel.load(f'trained_models/trained_lda_model_{nTopics}')

In [6]:
# Load topic distributions
topic_distributions = np.load(f'data/topic_distributions_{lda_model.num_topics}.npy')

In [8]:
# Pull topics
topics = lda_model.show_topics(formatted=False, num_topics=nTopics, num_words=20)

# Create Chord Diagrams

In [43]:
# Measure correlations
correlations = np.corrcoef(np.transpose(topic_distributions))# init storage
low_idx = np.where(np.abs(correlations) < 0.10)
correlations[low_idx] = 0

In [44]:
# Links for Chord Diagrams
links = pd.DataFrame(columns = ['source','target','value','n_value','p_value'])
row = -1
for i in range(topic_distributions.shape[1]):
    for j in range(i,topic_distributions.shape[1]):
        if not (i==j):
            row = row+1
            links.loc[row,'source'] = j    
            links.loc[row,'target'] = i
            links.loc[row,'n_value'] = np.max([0, -correlations[i,j]*100])
            links.loc[row,'p_value'] = np.max([0, correlations[i,j]*100])
links.value = links.p_value

# change data type
links = links.astype('int64')

In [45]:
# Nodes for Chord Diagrams
nodes = pd.DataFrame(columns = ['group','name'])
row = -1
for i in range(topic_distributions.shape[1]):
    row = row+1
    nodes.loc[row,'name'] = topic_names[i] 
    nodes.loc[row,'group'] = 0 

# change data type
nodes.group = nodes.group.astype('int64')

In [ ]:
hv.extension('bokeh')
hv.output(size = 200)

In [ ]:
# plot positive correlations
links.value = links.p_value
chord = hv.Chord((links, hv.Dataset(pd.DataFrame(nodes), 'index'))).select(value=(5, None))
chord.opts(
    opts.Chord(cmap='glasbey_light', 
               edge_cmap='glasbey_light', 
               edge_color=dim('source').str(), 
               labels='name', 
               node_color=dim('index').str()))

In [ ]:
import colorcet
# plot positive correlations
links.value = links.n_value
chord = hv.Chord((links, hv.Dataset(pd.DataFrame(nodes), 'index'))).select(value=(5, None))
chord.opts(
    opts.Chord(cmap='glasbey_light', 
               edge_cmap='glasbey_light', 
               edge_color=dim('source').str(), 
               labels='name', 
               node_color=dim('index').str()))